# WarZone Config Extractor

The config is stored in the `.bss` PE section with the following format.

Key length | key | data



In [1]:
# First lets setup our imports
import argparse
import struct
import pefile
import re
import binascii

Now a quick reminder of binary data hex encoding for Python3

In [3]:
string_example = "test"
byte_array_example = b"test"

# Convert string into bytes
print(string_example.encode('utf-8'))

# Convert byte array into string
print(byte_array_example.decode('utf-8'))

# Convert string into hex encoded byte array
print(binascii.hexlify(string_example.encode('utf-8')))

# Convert byte array into hex encoded byte array
print(binascii.hexlify(byte_array_example))

# Convert byte array into hex encoded string
print(binascii.hexlify(byte_array_example).decode('utf-8'))

# Convert hex encoded byte array into ascii byte array
hex_byte_array = b'74657374'
print(binascii.unhexlify(hex_string))

# Convert hex encoded string into ascii byte array
hex_string = '74657374'
print(binascii.unhexlify(hex_string.encode('utf-8')))

# Convert hex encoded string into ascii string
hex_string = '74657374'
print(binascii.unhexlify(hex_string.encode('utf-8')).decode('utf-8'))

b'test'
test
b'74657374'
b'74657374'
74657374
b'test'
b'test'
test


## Hex string helper functions

In [14]:
def unhex(hex_string):
    import binascii
    if type(hex_string) == str:
        return binascii.unhexlify(hex_string.encode('utf-8')).decode('utf-8')
    else:
        return binascii.unhexlify(hex_string).decode('utf-8')

def tohex(data):
    import binascii
    if type(data) == str:
        return binascii.hexlify(data.encode('utf-8')).decode('utf-8')
    else:
        return binascii.hexlify(data).decode('utf-8')
    

In [16]:

def unicode_strings(buf, n=4):
    import re
    ASCII_BYTE = b' !\"#\$%&\'\(\)\*\+,-\./0123456789:;<=>\?@ABCDEFGHIJKLMNOPQRSTUVWXYZ\[\]\^_`abcdefghijklmnopqrstuvwxyz\{\|\}\\\~\t'
    if type(data) == str:
        buf = buf.encode('utf-8')
    reg = b'((?:[%s]\x00){%d,})' % (ASCII_BYTE, n)
    uni_re = re.compile(reg)
    out = []
    for match in uni_re.finditer(buf):
        try:
            out.append(match.group().decode("utf-16"))
        except UnicodeDecodeError:
            pass
    return out


def ascii_strings(buf, n=4):
    import re
    ASCII_BYTE = b' !\"#\$%&\'\(\)\*\+,-\./0123456789:;<=>\?@ABCDEFGHIJKLMNOPQRSTUVWXYZ\[\]\^_`abcdefghijklmnopqrstuvwxyz\{\|\}\\\~\t'
    if type(data) == str:
        buf = buf.encode('utf-8')
    reg = b'([%s]{%d,})' % (ASCII_BYTE, n)
    ascii_re = re.compile(reg)
    out = []
    for match in ascii_re.finditer(buf):
        try:
            out.append(match.group().decode("ascii"))
        except UnicodeDecodeError:
            pass
    return out

## RC4 Encryption
The data is encrypted using RC4. For more information on RC4 check out our RC4 tutorial video.

<a href="https://www.youtube.com/watch?v=CiJocXXMXK4" target="_blank"><img src="http://img.youtube.com/vi/CiJocXXMXK4/0.jpg" 
alt="IMAGE ALT TEXT HERE" width="240" height="180" border="10" /></a>

In [4]:
# Now lets implement a simple RC4 decryption function
def rc4crypt(data, key):
    #If the input is a string convert to byte arrays
    if type(data) == str:
        data = data.encode('utf-8')
    if type(key) == str:
        key = key.encode('utf-8')
    x = 0
    box = list(range(256))
    for i in range(256):
        x = (x + box[i] + key[i % len(key)]) % 256
        box[i], box[x] = box[x], box[i]
    x = 0
    y = 0
    out = []
    for c in data:
        x = (x + 1) % 256
        y = (y + box[x]) % 256
        box[x], box[y] = box[y], box[x]
        out.append(c ^ box[(box[x] + box[y]) % 256])
    return bytes(out)

In [15]:
data = 'test'
key = 'key'
ctxt = rc4crypt(data, key)
print(ctxt)
ptxt = rc4crypt(ctxt, key)
print(ptxt)

b'\x7f\tG\x99'
b'test'
